In [1]:
import numpy as np

sample_size = 2518

with open('data.csv', 'w') as f:
    f.write('time,data\n')
    for i in range(sample_size):
        i -= sample_size / 2
        rad = np.deg2rad(2*i)
        y = np.sin(rad) + 100 * (i * i * i/ sample_size / sample_size / sample_size)
        f.write(f'{i},{y}\n')

In [2]:
import pandas as pd
import torch
print(torch.cuda.is_available())
from chronos import ChronosBoltPipeline

prediction_len = 192

pipeline = ChronosBoltPipeline.from_pretrained(
    "amazon/chronos-bolt-base",
    device_map="cuda",
    torch_dtype=torch.bfloat16,
)

df = pd.read_csv('data.csv')

True


# Using the model directly to predict, instead of through the pipeline

In [3]:
pipeline.model.to("cuda")
context_tensor = torch.tensor([df["data"]]).to("cuda")
forecast = torch.tensor([]).to('cpu')
for i in range(3):
    with torch.no_grad():
        pred = pipeline.model(context=context_tensor).quantile_preds
        avg = pred.mean(dim=1)
        forecast = torch.cat((forecast, avg[0].to('cpu')), dim=0)
        context_tensor = torch.cat((context_tensor, avg), dim=1)
m = torch.stack([
    torch.tensor([x + sample_size for x in range(prediction_len)]).float(),
    forecast
])

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [5]:
print(m[1])

tensor([12.6028, 12.6746, 12.7584, 12.8182, 12.8541, 12.9080, 12.9499, 13.0277,
        13.0756, 13.1294, 13.1534, 13.2611, 13.2551, 13.3209, 13.3628, 13.4107,
        13.4347, 13.4706, 13.5184, 13.5244, 13.5484, 13.6262, 13.6082, 13.6561,
        13.6741, 13.6800, 13.6860, 13.7339, 13.6980, 13.7040, 13.7040, 13.7279,
        13.7219, 13.7219, 13.6860, 13.7160, 13.6920, 13.6920, 13.6621, 13.6681,
        13.6800, 13.6381, 13.6381, 13.6441, 13.5783, 13.5903, 13.5244, 13.5484,
        13.5184, 13.4945, 13.5065, 13.5005, 13.4167, 13.3987, 13.3389, 13.3628,
        13.3569, 13.3209, 13.3030, 13.3030, 13.2192, 13.1713, 13.1115, 13.1055,
        13.2488, 13.2353, 13.1812, 13.1407, 13.0866, 13.0393, 12.9852, 12.9852,
        12.9785, 12.9447, 12.9041, 12.9312, 12.8839, 12.8366, 12.7825, 12.7555,
        12.7825, 12.7622, 12.7284, 12.7081, 12.6473, 12.6473, 12.5933, 12.6473,
        12.6541, 12.6338, 12.6068, 12.6000, 12.5324, 12.5189, 12.4784, 12.4987,
        12.5054, 12.4919, 12.4784, 12.49